In [ ]:


def q_resolution(wavelength_bins, moderator, data, q):
    
    l_collimation = 4.0*sc.units.m
    r2 = 0.004*sc.units.m
    r1 = 0.01*sc.units.m
    dr = 0.008*sc.units.m
    
    d_lam = wavelength_bins['wavelength', 1:] - wavelength_bins['wavelength', :-1] # bin widths
    lam = 0.5 * (wavelength_bins['wavelength', 1:] + wavelength_bins['wavelength', :-1]) # bin centres
    l2 = scn.L2(data)
    l1 = scn.L1(data)
    
    a1, a2, a3, wav_independent = get_wavelength_independent_factor(l2, l_collimation, r1, r2, dr)
    
    to_bin_edges(moderator, 'wavelength')
    moderatorValue = sc.rebin(moderator, 'wavelength', wavelength_bins)
    
    dq_sq = get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2)
    
    return a1, a2, a3, dq_sq 



In [ ]:


def get_aperture_smearing(l2, l_collimation, r1, r2, dr):
      
    inv_l3 = (l_collimation + l2) / (l_collimation * l2)
    
    # Terms in Mildner and Carpenter equation.
    # See https://docs.mantidproject.org/nightly/algorithms/TOFSANSResolutionByPixel-v1.html
    a1 = (r1/l_collimation)*(r1/l_collimation) * 3.0
    a2 = (r2*inv_l3)*(r2*inv_l3) * 3.0

    aperture_smearing = (4.0 * sc.constants.pi / 12) * (a1 + a2) 
    return aperture_smearing



In [ ]:
def get_detector_pixelation():
    
    a3 = (dr/l2) * (dr/l2) 
    return a3

In [ ]:


def get_sigma_q_value(data, d_lam, lam, wav_independent, moderatorValue, q, l1, l2):

     
    #sigModerator = Moderator time spread (microseconds) as afunction of wavelength (Angstroms)
    sig_moderator = moderatorValue * 3.9560 / (1000.0 * (l1 + l2));
    sig_moderator.unit = sc.units.angstrom
    
    #TODO: replace with q from cooridnate transformations 
    theta = scn.two_theta(data) 
    q_sq = 4.0 * sc.constants.pi * sc.sin(0.5*theta) * sc.reciprocal(lam)
    q_sq *= q_sq
    
    std_dev_lam_sq = q_sq 
    std_dev_lam_sq *= (d_lam * d_lam)/12 + sig_moderator * sig_moderator
    std_dev_lam_sq *= sc.reciprocal(lam * lam)
    
    std_dev_lam_independent = sc.DataArray((wav_independent * sc.reciprocal(lam * lam) ),
                    coords={'wavelength':std_dev_lam_sq.coords['wavelength']}) 
   
    dq_sq = sc.sqrt(std_dev_lam_independent + std_dev_lam_sq)
    return dq_sq   



In [ ]:
path = 'SANS2D_data'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
moderator = loki.io.load_rkh_wav(filename=f'{path}/{moderator_file}')

In [ ]:
l_collimation = 4.0*sc.units.m
r2 = 0.004*sc.units.m
r1 = 0.01*sc.units.m
dr = 0.008*sc.units.m

dq_sq = q_resolution(ds.coords['wavelength'], moderator, d, l_collimation, r1, r2, dr)

